**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}


Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` is the function that returns an action to pick, given the current state. It has two behaviors : 

- during the training phase of the agent, with probability `epsilon`, the agent will pick a random action, and with probability `1-epsilon`, it will pick the best value according to its learnt Q state action value function. The `epsilon` parameter is essential because it allows the agent to discover new states : this is known as the exploration phase, an important aspect of reinforcement learning algorithm.
- during the validation (=production) phase, we want to maximize the reward : we only pick optimal value (according to the Q function learnt by the agent during the training phase). This is known as the exploitation phase.

This function `act` shows a simple application of the exploration-exploitation trade-off in reinforcement learning.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=5 # set small when debugging
epochs_test=2 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The `position` array is used as the array that keeps track of the position of the rat, while the array `board` is used as the array of the rewards of being in every position (0 if nothing, 0.5 if cheese, and -1 if poisenous).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            action = agent.act(state, train=True) # Poor performance when train=False
            prev_state = state
            state, reward, game_over = env.act(action)
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=0.3)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/14.0. Average score (-4.0)
Win/lose count 14.5/20.0. Average score (-4.75)
Final score: -4.75


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




### **First equality**
Starting from the following equation : 

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

With $T = \infty$, we have that :

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] = r(s, a) + E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

Denoting by $p(.|s, a)$ the probability distribution of the next state-action given the current state action $(s, a)$, we have that :

\begin{equation*}Q^\pi(s,a)= r(s, a) + E_{(s', a') \sim p(.|s, a)}E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']
\end{equation*}

Which gives :

\begin{equation*}Q^\pi(s,a)= r(s, a) + \gamma E_{(s', a') \sim p(.|s, a)}E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']
\end{equation*}

But 

\begin{equation*}E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a'] = E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] = Q^{\pi}(s', a')
\end{equation*}

So that :

\begin{equation*}Q^\pi(s,a)= r(s, a) + \gamma E_{(s', a') \sim p(.|s, a)}Q^{\pi}(s', a')
\end{equation*}

And since $p(.|s, a)$ sums to one we can include the reward into the expectation :

\begin{equation*}Q^\pi(s,a)= E_{(s', a') \sim p(.|s, a)} [r(s, a) + \gamma Q^{\pi}(s', a')]
\end{equation*}

### **Second equality**
First we note that sampling $(s', a') \sim p(.|s, a)$ is equivalent to sampling $s'$ from $\tilde{p}(.|s, a)$ then $a'$ from $\pi(.|s')$.
We can suppose that the optimal policy is deterministic : we replace $a' \sim \pi(.|s')$ by $a' = \pi(s')$. Then by the equality we just proved, for $\pi = \pi^*$ :
$$
Q^*(s,a)= r(s, a) + E_{s' \sim \tilde{p}(.|s, a)} [\gamma Q^*(s', \pi^*(s')]
$$

But $Q^*(s', \pi^*(s')$ = $\max_{a'} Q^*(s', a')$ so we have : 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

### **Third part**
By the optimal Bellman equation, the error 
$$
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
$$
measures by how much our estimate $Q(., ., \theta)$ diverges from the ground truth $Q^*(s, a)$ : if $Q(., ., \theta) = Q^*(s, a)$, this error is 0 by the optimal Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.pos = 0

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            self.memory[self.pos % self.max_memory] = m
        else:
            self.memory.append(m)
        self.pos += 1

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        Qs_a = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        return Qs_a.argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1, 5, 5, self.n_state))
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount*self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model

        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(50, activation='relu'))
        # model.add(Dense(50, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging


env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/020 | Loss 0.0050 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/020 | Loss 0.0217 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/020 | Loss 0.0028 | Win/lose count 6.0/4.0 (2.0)
Epoch 003/020 | Loss 0.0051 | Win/lose count 6.0/7.0 (-1.0)
Epoch 004/020 | Loss 0.0087 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/020 | Loss 0.0015 | Win/lose count 1.5/5.0 (-3.5)
Epoch 006/020 | Loss 0.0021 | Win/lose count 1.5/3.0 (-1.5)
Epoch 007/020 | Loss 0.0080 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/020 | Loss 0.0025 | Win/lose count 3.5/2.0 (1.5)
Epoch 009/020 | Loss 0.0044 | Win/lose count 3.0/4.0 (-1.0)
Epoch 010/020 | Loss 0.0494 | Win/lose count 5.5/4.0 (1.5)
Epoch 011/020 | Loss 0.0033 | Win/lose count 3.5/4.0 (-0.5)
Epoch 012/020 | Loss 0.0026 | Win/lose count 4.5/0 (4.5)
Epoch 013/020 | Loss 0.0046 | Win/lose count 2.5/4.0 (-1.5)
Epoch 014/020 | Loss 0.0030 | Win/lose count 4.0/1.0 (3.0)
Epoch 015/020 | Loss 0.0014 | Win/lose count 6.5/4.0 (2.5)
Epoch 016/020 | Loss 0.0027 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(input_shape=(5, 5, self.n_state), filters=16, kernel_size=3, strides=1, activation='relu'))
        model.add(Conv2D(filters=16, kernel_size=3, strides=1, activation='relu'))
        model.add(Flatten())
        # model.add(Dense(50, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0014 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/020 | Loss 0.0039 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/020 | Loss 0.0072 | Win/lose count 6.5/3.0 (3.5)
Epoch 003/020 | Loss 0.0030 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/020 | Loss 0.0042 | Win/lose count 1.5/3.0 (-1.5)
Epoch 005/020 | Loss 0.0070 | Win/lose count 7.5/4.0 (3.5)
Epoch 006/020 | Loss 0.0252 | Win/lose count 5.5/0 (5.5)
Epoch 007/020 | Loss 0.0024 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/020 | Loss 0.0055 | Win/lose count 11.5/6.0 (5.5)
Epoch 009/020 | Loss 0.0031 | Win/lose count 13.5/6.0 (7.5)
Epoch 010/020 | Loss 0.0031 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/020 | Loss 0.0030 | Win/lose count 9.0/3.0 (6.0)
Epoch 012/020 | Loss 0.0035 | Win/lose count 12.0/4.0 (8.0)
Epoch 013/020 | Loss 0.0041 | Win/lose count 10.5/2.0 (8.5)
Epoch 014/020 | Loss 0.0013 | Win/lose count 4.5/0 (4.5)
Epoch 015/020 | Loss 0.0017 | Win/lose count 13.5/6.0 (7.5)
Epoch 016/020 | Loss 0.0023 | Win/lose count 13.0/3.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/5.0. Average score (-5.0)
Win/lose count 2.0/0. Average score (-1.5)
Win/lose count 5.0/3.0. Average score (-0.3333333333333333)
Win/lose count 1.0/3.0. Average score (-0.75)
Win/lose count 2.5/0. Average score (-0.1)
Win/lose count 1.5/3.0. Average score (-0.3333333333333333)
Win/lose count 2.0/0. Average score (0.0)
Win/lose count 3.0/0. Average score (0.375)
Win/lose count 4.5/2.0. Average score (0.6111111111111112)
Win/lose count 2.5/1.0. Average score (0.7)
Win/lose count 3.0/1.0. Average score (0.8181818181818182)
Win/lose count 1.0/0. Average score (0.8333333333333334)
Win/lose count 2.0/0. Average score (0.9230769230769231)
Win/lose count 1.5/0. Average score (0.9642857142857143)
Win/lose count 2.5/1.0. Average score (1.0)
Win/lose count 0/2.0. Average score (0.8125)
Win/lose count 2.0/2.0. Average score (0.7647058823529411)
Win/lose count 1.5/1.0. Average score (0.75)
Win/lose count 2.5/3.0. Average score (0.6842105263157895)
Win/lose count 1.5

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

I observe that both algorithms tend to get stuck in the neighborhood where they started and ate all the cheese, which hinders the full exploration of the space. This phenomenon is accentuated when the temperature decreases, as the reward signal becomes more sparse. This is in part due to the fact that the algorithms can only see at an horizon of 2 pixels. They should be incentivized to explore the space more. I also observe that the CNN based algorithms has better performance than the FC one.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='', max_epsilon=0.9, min_epsilon=0.1):
    # Number of won games
    score = 0
    loss = 0
    step_eps = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            reward -= env.malus_position[env.x, env.y]

            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            # Update the epsilon parameter
            epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-4*step_eps/(env.max_time * epoch))
            agent.set_epsilon(epsilon)
            step_eps += 1

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({}) | {:.3f}"
              .format(e, epoch, loss, win, lose, win-lose, epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        state, reward, game_over = super().act(action)
        self.malus_position[self.x, self.y] = 0.1
        state = np.concatenate((state[:, :, [0]], state[:, :, [1]], self.malus_position[self.x-2:self.x+3, self.y-2:self.y+3, None]), axis=2)
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        state = super().reset()
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        state = np.concatenate((state[:, :, [0]], state[:, :, [1]], self.malus_position[self.x-2:self.x+3, self.y-2:self.y+3, None]), axis=2)
        return state

In [0]:
# parameters
size = 13
T = 200


# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epoch=50, prefix='cnn_train_explore', max_epsilon=0.9, min_epsilon=0.1)
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0036 | Win/lose count 4.8/37.90000000000017 (-33.10000000000017) | 0.838
Epoch 001/050 | Loss 0.0064 | Win/lose count 11.600000000000005/25.200000000000056 (-13.600000000000051) | 0.781
Epoch 002/050 | Loss 0.0580 | Win/lose count 8.400000000000002/25.000000000000036 (-16.600000000000033) | 0.729
Epoch 003/050 | Loss 0.0356 | Win/lose count 8.800000000000002/30.900000000000112 (-22.100000000000108) | 0.680
Epoch 004/050 | Loss 0.0031 | Win/lose count 9.600000000000003/30.700000000000117 (-21.100000000000115) | 0.635
Epoch 005/050 | Loss 0.0056 | Win/lose count 11.200000000000005/27.30000000000011 (-16.100000000000108) | 0.594
Epoch 006/050 | Loss 0.0221 | Win/lose count 6.400000000000001/30.500000000000128 (-24.100000000000126) | 0.556
Epoch 007/050 | Loss 0.0043 | Win/lose count 15.200000000000008/27.300000000000068 (-12.10000000000006) | 0.521
Epoch 008/050 | Loss 0.0109 | Win/lose count 19.199999999999996/30.300000000000097 (-11.100000000000101) | 0.488
Epoch 

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 15.5/6.0. Average score (9.5)
Win/lose count 13.5/4.0. Average score (9.5)
Win/lose count 20.0/3.0. Average score (12.0)
Win/lose count 20.5/4.0. Average score (13.125)
Win/lose count 17.0/8.0. Average score (12.3)
Win/lose count 17.5/4.0. Average score (12.5)
Win/lose count 15.0/5.0. Average score (12.142857142857142)
Win/lose count 19.5/3.0. Average score (12.6875)
Win/lose count 19.0/4.0. Average score (12.944444444444445)
Win/lose count 15.0/2.0. Average score (12.95)
Win/lose count 19.5/4.0. Average score (13.181818181818182)
Win/lose count 17.5/4.0. Average score (13.208333333333334)
Win/lose count 17.5/1.0. Average score (13.461538461538462)
Win/lose count 22.5/3.0. Average score (13.892857142857142)
Win/lose count 21.0/4.0. Average score (14.1)
Win/lose count 21.5/3.0. Average score (14.375)
Win/lose count 25.0/2.0. Average score (14.882352941176471)
Win/lose count 17.0/3.0. Average score (14.833333333333334)
Win/lose count 17.0/1.0. Average score (14.89473684210

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***